# eg, t2g and dz component band and occupancy calculation
### By- <font color="orange">SOURAV GHOSH</font> 
#### Dept. of Physics, SRM AP
#### email- sourav_ghosh@srmap.edu.in
#### PhD Scholar (AP22135080004)
#### Guide- Prof. Ranjit Thapa

In [ ]:
import pandas as pd
import numpy as np
import math
from IPython.display import display

## READING AND VIEWING THE DATA

In [3]:
df= pd.read_csv('DOS53_7a_lorb11.csv')
# display(df.head())
# display(df.shape)
# display(df.columns)

## CHANGING THE COLUMN NAMES AND FILTERING THE DATA

In [28]:
##converting the energy column which is 'Unnamed: 0' to a numpy 1d array
energy=df['Unnamed: 0'][1:].values.reshape(-1,1).flatten()

##Finding the fermi energy level index
len_energy=len(energy)
fermi_level_index=0
for i in range(0,len_energy):
    if np.sign(energy[i])==1:
        fermi_level_index=i;
        break

## create a new dataframe by only splicing the required data here we need from Unnamed: 0 to Unnamed: 18
arr=[];
for i in range(19):
    arr.append(f'Unnamed: {i}')
#arr=['Unnamed: 0','Unnamed: 1','Unnamed: 2',...,'Unnamed: 17','Unnamed: 18']

filteredData=pd.DataFrame(df,columns=arr)
##slicing the new dataframe upto the fermi energy level--
## we have to add 1 with fermi level index because we have taken the energy array starting from 1st index as the 0th index has null
finalData=pd.DataFrame(filteredData[1:fermi_level_index+1]) 
finalData.rename(columns={'Unnamed: 0':'E',
                             'Unnamed: 1':'sUP',
                             'Unnamed: 2':'sDOWN',
                             'Unnamed: 3':'pyUP',
                             'Unnamed: 4':'pyDOWN',
                             'Unnamed: 5':'pzUP',
                             'Unnamed: 6':'pzDOWN',
                             'Unnamed: 7':'pxUP',
                             'Unnamed: 8':'pxDOWN',
                             'Unnamed: 9':'dxyUP',
                             'Unnamed: 10':'dxyDOWN',
                             'Unnamed: 11':'dyzUP',
                             'Unnamed: 12':'dyzDOWN',
                             'Unnamed: 13':'dz2UP',
                             'Unnamed: 14':'dz2DOWN',
                             'Unnamed: 15':'dxzUP',
                             'Unnamed: 16':'dxzDOWN',
                             'Unnamed: 17':'dx2-y2UP',
                             'Unnamed: 18':'dx2-y2DOWN',
                            },inplace = True) ##renaming the columns

finalData.index=range(0,finalData.shape[0])## we have to change the index otherwise it will start from 1
# finalData

## MAKING AND ADDING NEEDED COLUMNS TO THE DATA

In [29]:

##################################
####MAKING eg and t2g 
finalData['t2gUP']=finalData['dxyUP'].astype(float)+finalData['dxzUP'].astype(float)+finalData['dyzUP'].astype(float)
finalData['t2gDOWN']=finalData['dxyDOWN'].astype(float)+finalData['dxzDOWN'].astype(float)+finalData['dyzDOWN'].astype(float)
finalData['t2gTOTAL']=finalData['t2gUP']-finalData['t2gDOWN']
finalData['egUP']=finalData['dz2UP'].astype(float)+finalData['dx2-y2UP'].astype(float)
finalData['egDOWN']=finalData['dz2DOWN'].astype(float)+finalData['dx2-y2DOWN'].astype(float)
finalData['egTOTAL']=finalData['egUP']-finalData['egDOWN']

###################################
##MAKING Z COMPONENT
finalData['dzcompUP']=finalData['dyzUP'].astype(float)+finalData['dxzUP'].astype(float)+finalData['dz2UP'].astype(float)
finalData['dzcompDOWN']=finalData['dyzDOWN'].astype(float)+finalData['dxzDOWN'].astype(float)+finalData['dz2DOWN'].astype(float)
finalData['dzcompTOTAL']=finalData['dzcompUP']-finalData['dzcompDOWN']
#################################
#####MAKING  n*E

orbitalArray=['eg','t2g','dzcomp']
iterArray=['UP','DOWN','TOTAL']

    
for i in range(0,len(orbitalArray)):
    for j in range(len(iterArray)):
        n=finalData[f'{orbitalArray[i]}{iterArray[j]}'].astype(float)
        E=finalData['E'].astype(float)
        finalData[f'{orbitalArray[i]}{iterArray[j]}*E']=n*E 
    
# finalData

# RESULTS

In [34]:
####################
####### RESULTS    #
#################### 

def integrate(x, y):
    area = np.trapz(y=y, x=x)
    return area

#######################################################################################################################
# UNCOMMENT IT TO PRINT THE RESULTS                                                                                   #
######                                                                                                                #
# for i in range(len(orbitalArray)):                                                                                  #
#     for j in range(len(iterArray)):                                                                                 #
#         integration_nE=integrate(finalData[f'{orbitalArray[i]}{iterArray[j]}'].values,finalData['E'].values)        #
#         integration_EnE=integrate(finalData[f'{orbitalArray[i]}{iterArray[j]}*E'].values,finalData['E'].values)     #
#         print(f'Integration of {orbitalArray[i]}{iterArray[j]} is:: {integration_EnE/integration_nE} ')             #
#         print(f'Occupancy of {orbitalArray[i]}{iterArray[j]} is::{integration_nE}')                                 #
#######################################################################################################################
        
result={}        
for i in range(len(orbitalArray)):
    result[i]={orbitalArray[i]:{}}
    for j in range(len(iterArray)):
        integration_nE=integrate(finalData[f'{orbitalArray[i]}{iterArray[j]}'].values,finalData['E'].values)
        integration_EnE=integrate(finalData[f'{orbitalArray[i]}{iterArray[j]}*E'].values,finalData['E'].values)
        result[i][orbitalArray[i]][iterArray[j]]={'rho*E':integration_EnE/integration_nE,'occupancy':integration_nE}
        
# print(result)

iterables = [orbitalArray, iterArray]
index=pd.MultiIndex.from_product(iterables) ##, names=["first", "second"]##)
resultDF=pd.DataFrame(index=index)
rhoElist=[]
occupancylist=[]
for i in result:
    for j in result[i]:
        for k in result[i][j]:
            for l in result[i][j][k]:
                if l=='rho*E':
                    rhoElist.append(result[i][j][k][l])
                else:
                    occupancylist.append(result[i][j][k][l])

                                 
# print(rhoElist)
# print(occupancylist)
resultDF['rho*E']=rhoElist
resultDF['occupancy']=occupancylist

resultDF.to_csv('result.csv')
resultDF

rho*E  occupancy
eg     UP    -1.599790  -1.833743
       DOWN  -1.084651   1.763643
       TOTAL -1.347240  -3.597385
t2g    UP    -2.593562  -2.247177
       DOWN  -2.478429   1.463543
       TOTAL -2.548153  -3.710720
dzcomp UP    -1.607201  -2.672973
       DOWN  -1.014261   1.868881
       TOTAL -1.363218  -4.541855